In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
from hyperopt import fmin, tpe, hp
from functools import partial
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unrar x /content/drive/MyDrive/data_images.rar

In [9]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
class vision_model(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(vision_model, self).__init__()
        self.tmodel = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
        for param in self.tmodel.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(1000, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.dropout(self.tmodel(x))
        x = self.fc1(x)
        return x

# Define the training function
def train(model, criterion, optimizer, train_loader, device):
    model.train()
    train_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        correct += (torch.argmax(output, 1) == torch.argmax(target, 1)).float().sum().item()
    train_loss /= len(train_loader.dataset)
    acc = correct / len(train_loader.dataset)
    return train_loss, acc

# Define the validation function
def validate(model, criterion, val_loader, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            correct += (torch.argmax(output, 1) == torch.argmax(target, 1)).float().sum().item()
            #correct += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

# Define the objective function for HyperOpt
def objective(params, model, criterion, optimizer, train_loader, val_loader, device):
    dropout_rate = params['dropout_rate']
    lr = params['lr']
    model.dropout.p = dropout_rate
    optimizer.lr = lr
    train_loss, c = train(model, criterion, optimizer, train_loader, device)
    val_loss, accuracy = validate(model, criterion, val_loader, device)
    return {'loss': val_loss, 'status': 'ok', 'accuracy': accuracy}

# Define the hyperparameter space
space = {
    'dropout_rate': hp.uniform('dropout_rate', 0, 1),
    'lr': hp.loguniform('lr', -6, -1)
}

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [11]:
class Dataset_Image(Dataset):
    def __init__(self, data_path, transform=None, test=False):
        self.dataset = datasets.ImageFolder(data_path, transform=transform)
        self.num_classes = len(self.dataset.classes)
        self.clases = self.dataset.classes
        self.test = test

    def __getitem__(self, index):
        
        image, label = self.dataset[index]
        one_hot_label = torch.zeros(self.num_classes)
        one_hot_label[label] = 1
        if self.test:
          return image
        return image, one_hot_label

    def __len__(self):
        return len(self.dataset)

In [12]:
# load the training and validation datasets
train_dataset = Dataset_Image('/content/train', transform=transform)
val_dataset = Dataset_Image('/content/val', transform=transform)

# create data loaders for the datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

print (len(train_loader), len(val_loader))

88 7


In [13]:
print(train_dataset.clases)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [14]:
model = vision_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

partial_objective = partial(objective, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_loader, val_loader=val_loader, device=device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 93.7MB/s]


In [17]:
best = fmin(fn=partial_objective, space=space, algo=tpe.suggest, max_evals=10)

  0%|          | 0/10 [03:04<?, ?trial/s, best loss=?]


KeyboardInterrupt: ignored

In [ ]:
print("Best hyperparameters:", best)
best_model = vision_model(dropout_rate=best['dropout_rate']).to(device)
best_optimizer = optim.Adam(best_model.parameters(), lr=best['lr'])

In [ ]:
best_model_acc = 0
for x in range(5):
  loss_train, acc = train(best_model, nn.CrossEntropyLoss(), best_optimizer, train_loader, device)
  best_val_loss, best_accuracy = validate(best_model, criterion, val_loader, device)
  print ('Epochs {} ---- Train Loss: {} ---- Acuracy Train {} ---- Val Loss {} ---- Accuracy Val {}'.format(x+1, loss_train, acc, best_val_loss, best_accuracy))
  if best_accuracy > best_model_acc:
    torch.save(best_model.state_dict(), "model.pth")

In [ ]:
torch.save(best_model, "model.pt")

In [ ]:
# Define the validation function
def predict(model, val_loader, device):
    model.eval()
    correct = 0
    prediction = []
    true = []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            correct += (torch.argmax(output, 1) == torch.argmax(target, 1)).float().sum().item()
            prediction.extend(torch.argmax(output, 1).cpu().numpy())
            true.extend(torch.argmax(target, 1).cpu().numpy())
    accuracy = correct / len(val_loader.dataset)
    return accuracy, prediction, true
model_0 = torch.load("/content/model.pt")
#print (predict(model_0, train_loader, device))
a, b, c = predict(model_0, val_loader, device)

In [ ]:
predicted_label = [train_dataset.clases[x] for x in b]
true_label = [train_dataset.clases[x] for x in c]

In [ ]:
confusion_mtx = confusion_matrix(predicted_label, true_label, labels=train_dataset.clases)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx, display_labels=train_dataset.clases)
fig, ax = plt.subplots(figsize=(12,10))
disp.plot(ax=ax)
plt.show()

#['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print ("Number of correct Airplane : {} --- Number of Wrong Airplane : {} --- Airplane Accuracy: {} ".format(confusion_mtx[:, 0][0], np.sum(confusion_mtx[:, 0][1:]), confusion_mtx[:, 0][0] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Automobile : {} --- Number of Wrong Automobile : {} --- Automobile Accuracy: {} ".format(confusion_mtx[:, 0][1], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][1], confusion_mtx[:, 0][1] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Bird : {} --- Number of Wrong Bird : {} --- Bird Accuracy: {} ".format(confusion_mtx[:, 0][2], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][2], confusion_mtx[:, 0][2] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Cat : {} --- Number of Wrong Cat : {} --- Cat Accuracy: {} ".format(confusion_mtx[:, 0][3], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][3], confusion_mtx[:, 0][3] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Deer : {} --- Number of Wrong Deer : {} --- Deer Accuracy: {} ".format(confusion_mtx[:, 0][4], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][4], confusion_mtx[:, 0][4] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Dog : {} --- Number of Wrong Dog : {} --- Dog Accuracy: {} ".format(confusion_mtx[:, 0][5], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][5], confusion_mtx[:, 0][5] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Frog : {} --- Number of Wrong Frog : {} --- Frog Accuracy: {} ".format(confusion_mtx[:, 0][6], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][6], confusion_mtx[:, 0][6] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Horse : {} --- Number of Wrong Horse : {} --- Horse Accuracy: {} ".format(confusion_mtx[:, 0][7], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][7], confusion_mtx[:, 0][7] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Ship : {} --- Number of Wrong Ship : {} --- Ship Accuracy: {} ".format(confusion_mtx[:, 0][8], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][8], confusion_mtx[:, 0][8] / np.sum(confusion_mtx[:, 0])))

print ("Number of correct Truck : {} --- Number of Wrong Truck : {} --- Truck Accuracy: {} ".format(confusion_mtx[:, 0][9], np.sum(confusion_mtx[:, 0]) - confusion_mtx[:, 0][9], confusion_mtx[:, 0][9] / np.sum(confusion_mtx[:, 0])))

In [ ]:
example_input = torch.rand(1, 3, 224, 224).to(device)

traced_script_module = torch.jit.trace(model_0, example_input)
traced_script_module.save("model_torchscipt.pt")